This notebook aims to build a graph purely with KNN.

In [1]:
import numpy as np
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/mini_nusc', verbose=True)

# nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/nuscenes2', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.630 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [2]:
import torch_geometric as tgeo

tgeo.get_home_dir()

'/home/maximilian/.cache/pyg'

In [3]:
from nuscenes.utils.splits import train,val
from nuscenes.utils.splits import create_splits_scenes

split = create_splits_scenes()
print(split.keys())
split_scene_list = []
for scene_name in split['mini_train']:
    for scene in nusc.scene:
        if scene['name']==scene_name:
            split_scene_list.append(scene)

dict_keys(['train', 'val', 'test', 'mini_train', 'mini_val', 'train_detect', 'train_track'])


In [4]:
#write a sequence 

sample_dict = {}

i = 0 
for scene in split_scene_list:
    last_sample_token =""
    sample_token = scene['first_sample_token']
    while(last_sample_token == ""):
        
        sample = nusc.get('sample', sample_token)
        sample_dict[i] = (scene['token'],sample["token"])
        i += 1
        sample_token = sample["next"]
        if(sample["token"]== scene['last_sample_token']):
            last_sample_token = scene['last_sample_token']

print (len(sample_dict))
print (sample_dict[0])
print (sample_dict[1])
print (sample_dict[2])

323
('cc8c0bf57f984915a77078b10eb33198', 'ca9a282c9e77460f8360f564131a8af5')
('cc8c0bf57f984915a77078b10eb33198', '39586f9d59004284a7114a68825e8eec')
('cc8c0bf57f984915a77078b10eb33198', '356d81f38dd9473ba590f39e266f54e5')


In [5]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.is_available())
    num_gpu = torch.cuda.device_count()
    print("num_gpu:",num_gpu)
    print("current_device: ",torch.cuda.current_device())
    print("---------------------------------------------------------------")
    for i in range(num_gpu):
        print(torch.cuda.get_device_name(i))
        # print(torch.cuda.device(i))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB', torch.cuda.memory_allocated(i),torch.cuda.memory_allocated(i)/1024**3)
        print('Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB', torch.cuda.memory_reserved(i),torch.cuda.memory_reserved(i)/1024**3)
        print("---------------------------------------------------------------")

Using device: cuda
True
num_gpu: 2
current_device:  0
---------------------------------------------------------------
GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB 0 0.0
Cached:    0.0 GB 0 0.0
---------------------------------------------------------------
GeForce RTX 2080
Memory Usage:
Allocated: 0.0 GB 0 0.0
Cached:    0.0 GB 0 0.0
---------------------------------------------------------------


In [16]:
import torch
import numpy as np

a = torch.rand((3,4))
b = torch.empty(6,4, dtype=torch.int8)

print(a,"\n", b)

b[0:,:]=a
print(b)

# from enum import IntEnum
# class edge_types(IntEnum):
#     spatial_edges = 0
#     temporal_edges = 1

# y = torch.LongTensor(4,1).random_() % 2
# print("y:", y)
# y_onehot = torch.FloatTensor(4, 2)
# y_onehot.zero_()
# y_onehot.scatter_(1,y,1)
# print("y_onehot:", y_onehot)

# from sklearn.neighbors import BallTree
# rng = np.random.RandomState(0)
# X = rng.random_sample((10, 3))
# tree = BallTree(X, leaf_size=2) 
# dist, ind = tree.query(X, k=3) 
# print(ind)  # indices of 3 closest neighbors
# ind = ind[:,1:]
# print(ind) 

tensor([[0.5267, 0.4764, 0.4718, 0.4794],
        [0.6225, 0.9324, 0.5122, 0.6801],
        [0.3192, 0.7583, 0.7667, 0.3306]]) 
 tensor([[ 32,  93,   2, -85],
        [ -1, 126,   0,   0],
        [112, -60,   2, -58],
        [ -2,  85,   0,   0],
        [ 32,   0,   0,   0],
        [  0,   0,   0,   0]], dtype=torch.int8)


RuntimeError: The expanded size of the tensor (6) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [6, 4].  Tensor sizes: [3, 4]

In [7]:
#Create List of Graph objects
from datasets.nuscenes_mot_graph import NuscenesMotGraph
#______________________________________________________________#
# Decide if only first scene should be computed
only_first_scene = True
scene_token0, sample_token0= sample_dict[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    for i in range(torch.cuda.device_count()):
        if torch.cuda.get_device_name(i) == "GeForce RTX 2080":
            device = torch.device(i) # RTX 2080 8GB
print("Device:", device)
#_______________________________________________________________#

MotGraphList= []
for sample_key in sample_dict:
    scene_token_current, sample_token_current= sample_dict[sample_key]
    if(only_first_scene):
        if(scene_token0 == scene_token_current):
            object = NuscenesMotGraph(nuscenes_handle = nusc,
                        start_frame=sample_token_current,
                        max_frame_dist = 3, 
                        filterBoxes_categoryQuery='vehicle.car',
                        device= device)
            is_possible2construct = object.is_possible2construct
            if is_possible2construct:
                object.construct_graph_object()
                # object.assign_edge_labels_one_hot()
                object.assign_edge_labels(label_type='multiclass')
                # object.assign_edge_labels(label_type='binary')
                MotGraphList.append(object)
    else:
        object = NuscenesMotGraph(nuscenes_handle = nusc,
                        start_frame=sample_token_current,
                        max_frame_dist = 3,  
                        filterBoxes_categoryQuery='vehicle.car',device= device)
        is_possible2construct = object.is_possible2construct
        if is_possible2construct:
            object.construct_graph_object()
            object.assign_edge_labels(label_type='binary')
            MotGraphList.append(object)
            

Device: cuda:1


In [8]:
# # Input
# print("----------------------------------------\n")
# print(MotGraphList[0].graph_obj.keys)
# print("----------------------------------------\n")
# print("Node features: ", MotGraphList[0].graph_obj['x'])
# print("----------------------------------------\n")
# print("Edge indices", MotGraphList[0].graph_obj['edge_index'])
# print("----------------------------------------\n")
# print("Edge attributes (only temporal until now)", MotGraphList[0].graph_obj['edge_attr'])

In [9]:
# #Output
# from groundtruth_generation.nuscenes_create_gt import edge_label_classes

# print("edge_label_classes.different_instance:",edge_label_classes.different_instance)
# print("edge_label_classes.same_instance:",edge_label_classes.same_instance)
# print("edge_label_classes.new_instance:",edge_label_classes.new_instance)
# print("Edge labels: one hot encoding",MotGraphList[0].graph_obj["edge_labels"])

In [9]:
print("----------------------------------------\n")
print(len(MotGraphList))
print("----------------------------------------\n")
print(MotGraphList[0])
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj)
print("----------------------------------------\n")
print(len(MotGraphList[0].graph_obj))
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj.keys)
print("----------------------------------------\n")
print("is_undirected() : ",MotGraphList[0].graph_obj.is_undirected())
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj["x"][0:5])

print("----------------------------------------\n")
print('graph_dataframe["boxes_dict"][1][0] \n',MotGraphList[0].graph_dataframe["boxes_dict"][1][0])
print("----------------------------------------\n")
box_list, center_list = MotGraphList[0].graph_dataframe["centers_dict"][1]
print("box_list: \n",box_list[0], 'center_list: \n', center_list[0])
print("----------------------------------------\n")
box_list, center_list = MotGraphList[0].graph_dataframe["centers_dict"][1]
print("is same memory ?: \n", MotGraphList[0].graph_dataframe["boxes_dict"][1] is box_list )
print("----------------------------------------\n")
for i in range(3):
    print(MotGraphList[0].graph_dataframe["boxes_list_all"][i] )
    print(MotGraphList[0].graph_dataframe["centers_list_all"][i])
    print("----------------------------------------\n")
print("----------------------------------------\n")
for i in range(3):
    print(MotGraphList[0].graph_dataframe["boxes_list_all"][-i] )
    print(MotGraphList[0].graph_dataframe["centers_list_all"][-i])
    print("----------------------------------------\n")
print("----------------------------------------\n")
print("Edge Labels: ",MotGraphList[0].graph_obj["edge_labels"])
print("----------------------------------------\n")
print("Edge Labels: ",MotGraphList[-1].graph_obj["edge_labels"])
print("----------------------------------------\n")

print("Nodes shape: ",MotGraphList[0].graph_obj.x.shape)
print("----------------------------------------\n")
print("Edge_index : ",MotGraphList[0].graph_obj["edge_index"])
print("----------------------------------------\n")

----------------------------------------

37
----------------------------------------

----------------------------------------

Graph(x=[29, 3], edge_index=[2, 330], edge_attr=[330, 2], temporal_edges_mask=[330, 2], edge_labels=[330, 3])
----------------------------------------

5
----------------------------------------

['edge_attr', 'edge_labels', 'x', 'temporal_edges_mask', 'edge_index']
----------------------------------------

is_undirected() :  True
----------------------------------------

tensor([[ 3.7352e+01,  6.4397e+01,  4.5099e-01],
        [ 9.1482e+00, -1.9542e+01, -1.6450e+00],
        [ 5.9793e+00,  3.5009e+01,  4.4059e-02],
        [-8.2717e+00,  7.7670e+01,  2.0498e+00],
        [ 3.3012e+00,  4.0340e+01,  1.4580e-01]], device='cuda:1',
       dtype=torch.float64)
----------------------------------------

graph_dataframe["boxes_dict"][1][0] 
 label: nan, score: nan, xyz: [37.71, 59.68, 0.38], wlh: [2.01, 4.63, 1.57], rot axis: [0.01, -0.02, 1.00], ang(degrees): 176.

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(3, 16)
        self.conv2 = GCNConv(16, 2)
        self.mlp = torch.nn.Linear(
                    in_features=3, out_features=1, bias=True)

    def forward(self, data):
        x = data.x 
        edge_index = data.edge_index
        edge_feature =data.edge_attr

        # x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = self.conv2(x, edge_index)
        x = x.float()
        x = self.mlp(x)

        return F.sigmoid(x)

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

graph_list = []
for graph in MotGraphList:
    graph_list.append(graph.graph_obj)
print(len(graph_list))
print(graph_list[:5])
train_loader = DataLoader(graph_list, batch_size=1,shuffle =False)

In [ ]:
# for data in train_loader:
    # print(data.x)
    # print(data.edge_index)
    # print(data.edge_attr)
    # print(data.num_graphs)

# a = torch.zeros(3,5).to('cuda')
# probabilities_labels = torch.empty(a.shape).uniform_(0, 1).to('cuda')
# label = torch.bernoulli(probabilities_labels).to('cuda')
# print(a)
# print(probabilities_labels)
# print(label)

In [ ]:

# def train():
#     model.train()

#     loss_all = 0
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         label = data.y.to(device)
#         loss = crit(output, label)
#         loss.backward()
#         loss_all += data.num_graphs * loss.item()
#         optimizer.step()
#     return loss_all / len(graph_list)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = Net().to(device)
print(next(model.parameters()).is_cuda)
# print(model.parameters())

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()

print(train_loader)

loss_general = []
for epoch in range(4):
    model.train()
    loss_all = 0
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        #Generate random labels for testing pipeline
        # generate binary labels 
        probabilities_labels = torch.empty(output.shape).uniform_(0, 1).to(device)
        node_labels_binary = torch.bernoulli(probabilities_labels)
        loss = crit(output, node_labels_binary)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    loss_normalized =loss_all / len(graph_list)
    loss_general.append(loss_normalized)
    print('epoch',epoch,'loss_general:',loss_normalized)

In [ ]:
import tqdm